In [21]:
# Install all dependencies
!pip install -q transformers torchaudio moviepy fpdf nltk


In [22]:
!python -m nltk.downloader popular

<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_dat

In [23]:
# Imports
import os
import nltk
# nltk.download('punkt')

from nltk.tokenize import sent_tokenize
from transformers import pipeline
from moviepy.editor import VideoFileClip
from fpdf import FPDF
from google.colab import files


In [24]:
# Load Pegasus summarizer
text_summarizer = pipeline("summarization", model="google/pegasus-xsum")

# Chunking logic
def chunk_text(text, max_tokens=900):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = ""
    current_tokens = 0

    for sentence in sentences:
        tokens = len(sentence.split())
        if current_tokens + tokens <= max_tokens:
            current_chunk += sentence + " "
            current_tokens += tokens
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
            current_tokens = tokens

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

# Text summarization logic
def summarize_text(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()

    chunks = chunk_text(text, max_tokens=900)

    chunk_summaries = [
        text_summarizer(chunk, max_length=200, min_length=80, do_sample=False)[0]['summary_text']
        for chunk in chunks
    ]

    detailed_summary = "\n".join(chunk_summaries)
    short_summary = text_summarizer(detailed_summary, max_length=80, min_length=30, do_sample=False)[0]['summary_text']

    return short_summary, detailed_summary

# Export summary
def export_summary(short_summary, detailed_summary, output_path="summary_output.pdf"):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 10, f"Short Summary:\n{short_summary}\n\nDetailed Summary:\n{detailed_summary}")
    pdf.output(output_path)
    files.download(output_path)


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


In [25]:
def summarize_video(video_path):
    # Step 1: Extract audio
    clip = VideoFileClip(video_path)
    clip.audio.write_audiofile("extracted_audio.mp3")

    # Step 2: Transcribe with Whisper
    whisper_pipe = pipeline(model="openai/whisper-base")
    transcription = whisper_pipe("extracted_audio.mp3", return_timestamps=True)
    print("\n📝 Transcription:\n")
    print(transcription["text"])

    # Step 3: Summarize
    summarization = pipeline('summarization')
    summary = summarization(transcription['text'])
    summary = summary[0]['summary_text']
    print("Summary of video is:", summary)

    # Step 4: Export
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 10, f"Summary of the Video:\n\n{summary}")
    pdf.output("video_summary.pdf")
    files.download("video_summary.pdf")


In [29]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [30]:
# Upload file
uploaded = files.upload()
file_path = list(uploaded.keys())[0]
ext = os.path.splitext(file_path)[1].lower()

if ext == ".txt":
    print("📄 Text file detected. Running text summarizer...")
    short_summary, detailed_summary = summarize_text(file_path)
    export_summary(short_summary, detailed_summary)

elif ext == ".mp4":
    print("🎥 Video file detected. Running video summarizer...")
    summarize_video(file_path)

else:
    print("❌ Unsupported file format. Please upload a .txt or .mp4 file.")


Saving APJ.txt to APJ (6).txt
📄 Text file detected. Running text summarizer...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>